Source: https://medium.com/@lope.ai/recommendation-systems-from-scratch-in-python-pytholabs-6946491e76c2

In [1]:
import pandas as pd
import numpy as np

In [2]:
gourmet_foods = pd.read_csv('./gourmet_foods.csv')

C:\Users\Suchi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
gourmet_foods.head(5)

,overall,reviewerID,asin,reviewerName,reviewText,summary,vote
0,5.0,A1QVBUH9E1V6I8,4639725183,Jamshed Mathur,No adverse comment.,Five Stars,NaN
1,5.0,A3GEOILWLK86XM,4639725183,itsjustme,Gift for college student.,Great product.,NaN
2,5.0,A32RD6L701BIGP,4639725183,Krystal Clifton,"If you like strong tea, this is for you. It mi...",Strong,NaN
3,5.0,A2UY1O1FBGKIE6,4639725183,U. Kane,Love the tea. The flavor is way better than th...,Great tea,NaN
4,5.0,A3QHVBQYDV7Z6U,4639725183,The Nana,I have searched everywhere until I browsed Ama...,This is the tea I remembered!,NaN


In [4]:
gourmet_foods = gourmet_foods.rename(columns={"asin": "productID"})

In [5]:
gf = pd.DataFrame({"reviewerID": gourmet_foods['reviewerID'], "productID": gourmet_foods['productID'],
                  "rating": gourmet_foods['overall']})

In [6]:
gf.head(5)

,reviewerID,productID,rating
0,A1QVBUH9E1V6I8,4639725183,5.0
1,A3GEOILWLK86XM,4639725183,5.0
2,A32RD6L701BIGP,4639725183,5.0
3,A2UY1O1FBGKIE6,4639725183,5.0
4,A3QHVBQYDV7Z6U,4639725183,5.0


In [7]:
gf

,reviewerID,productID,rating
0,A1QVBUH9E1V6I8,4639725183,5.0
1,A3GEOILWLK86XM,4639725183,5.0
2,A32RD6L701BIGP,4639725183,5.0
3,A2UY1O1FBGKIE6,4639725183,5.0
4,A3QHVBQYDV7Z6U,4639725183,5.0
...,...,...,...
993842,A223YRQH2Z5T1D,B01HJF6FRA,5.0
993843,A38GDA4TB9EILT,B01HJF6FRA,5.0
993844,A2025PN7HDC5BO,B01HJF6FRA,5.0
993845,A1NY7XWC7EPQOA,B01HJF6FRA,5.0


In [8]:
to_csv = gf.to_csv('./gourmet_food_ratings.csv', index=False)

In [9]:
gf_sampled = gf.sample(20000)

In [10]:
gf_num_users = gf_sampled['reviewerID'].unique().shape[0]
gf_num_products = gf_sampled['productID'].unique().shape[0]

In [11]:
gf_sampled

,reviewerID,productID,rating
146576,A2S696CI415O20,B000UYFKRY,5.0
134749,AO97AQNALDTQC,B000QSS23S,5.0
484482,A2WIMBIRP0SKQQ,B0068L0WX0,5.0
1708,A1NSOZH0EYGA2W,B0000D9169,5.0
609165,A3JAKR6IPAKQTD,B00CXXK4EO,5.0
...,...,...,...
3550,A2JAWYBEW9MWT6,B0000EYP2U,3.0
933707,A8AMLLYIF68IS,B005K4Q4LK,5.0
248763,A241OH2YXYHU3C,B001A3EDAO,2.0
579515,A3KOSRR1P7WIH2,B00BQWU5OW,5.0


In [12]:
gf_user_ids = dict()
gf_users = list(gf_sampled['reviewerID'].unique())
for i,user in enumerate(gf_users):
    gf_user_ids[user] = i

In [13]:
gf_product_ids = dict()
gf_products = list(gf_sampled['productID'].unique())
for j,product in enumerate(gf_products):
    gf_product_ids[product] = j

In [14]:
reviewer_ids = list()
for reviewer in gf_sampled['reviewerID']:
    reviewer_ids.append(gf_user_ids[reviewer])

In [15]:
product_ids = list()
for product in gf_sampled['productID']:
    product_ids.append(gf_product_ids[product])

In [16]:
gf_df = pd.DataFrame({'reviewerID': reviewer_ids, 'productID': product_ids, 'rating': gf_sampled['rating']})

In [17]:
data_matrix = np.zeros((gf_num_users, gf_num_products))
for line in gf_df.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]

In [18]:
gf_num_products

10663

In [19]:
from sklearn.metrics.pairwise import pairwise_distances

In [20]:
user_similarity = pairwise_distances(data_matrix, metric='cosine')

In [21]:
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [22]:
def predict(ratings, similarity, type='reviewer'):
    
    if type == 'reviewer':
        mean_reviewer_rating = ratings.mean(axis=1).reshape(-1,1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        
        ratings_diff = (ratings - mean_reviewer_rating)
        pred = mean_reviewer_rating + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    
    elif type == 'product':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    
    return pred

In [23]:
user_prediction = predict(data_matrix, user_similarity, type='reviewer') #do it for each chunk
item_prediction = predict(data_matrix, item_similarity, type='product') #do it for each chunk

In [24]:
def label_sentiment(row):
    cutoff = .05
    if row.compound < -1 * cutoff:
        return "neg"
    elif row.compound > cutoff:
        return "pos"
    else:
        return "neu"

In [25]:
def sentiment_analysis(productID):
    review_text = gourmet_foods[gourmet_foods['productID'] == productID]['reviewText']
    sia = SentimentIntensityAnalyzer()
    sws = set(stopwords.words('english'))
    processed_text = list()

    for review in review_text:
        review = review.translate(review.maketrans('', '', string.punctuation))
        review = review.lower()
        no_sw_sentences = list()
        sentences = nltk.sent_tokenize(review)
        for sentence in sentences:
            no_sw_words = list()
            words = nltk.word_tokenize(sentence)
            for word in words:
                if word not in sws:
                    no_sw_words += [word]
            no_sw_sentences += [' '.join(no_sw_words)]
        if(len(no_sw_sentences) == 0):
            no_sw_sentences = ''
        else:
            temp2 = no_sw_sentences[0]
            no_sw_sentences = str(temp2)
        processed_text.append(no_sw_sentences)
    
    scores = []
    for sentence in processed_text:
        data = sid.polarity_scores(sentence)
        data['text'] = sentence
        scores += [data]
    
    df_scores = pd.DataFrame(scores)
    df_scores['labels'] = df_scores.apply(label_sentiment, axis=1)
    
    df_scores
    